In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import os
import pandas as pd

In [ ]:
with open('train_annotations.json', 'r') as f:
    data = json.load(f)

In [ ]:
print(data.keys())

In [ ]:
images_df = pd.DataFrame(data['images'])
categories_df = pd.DataFrame(data['categories'])
annotations_df = pd.DataFrame(data['annotations'])

In [ ]:
images_df.head()

In [ ]:
categories_df

In [ ]:
annotations_df

In [ ]:
merged_df = annotations_df.merge(images_df, left_on='image_id', right_on='id', suffixes=('_ann', '_img'))
merged_df.head()

In [ ]:
merged_df = merged_df.merge(categories_df, left_on='category_id', right_on='id', suffixes=('', '_cat'))
merged_df.head()

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

In [ ]:
IMAGES_FOLDER = 'eccv_18_all_images_sm'

In [ ]:
sample = merged_df.iloc[10]
image_path = os.path.join(IMAGES_FOLDER, sample['file_name'])

img = Image.open(image_path)
draw = ImageDraw.Draw(img)

In [ ]:
original_width = sample['width']
original_height = sample['height']

scale_x = img.width / original_width
scale_y = img.height / original_height

bbox = sample['bbox']
x0, y0, width, height = map(float, bbox)
x1, y1 = x0 + width, y0 + height

x0_scaled = x0 * scale_x
y0_scaled = y0 * scale_y
x1_scaled = x1 * scale_x
y1_scaled = y1 * scale_y

draw.rectangle([x0_scaled, y0_scaled, x1_scaled, y1_scaled], outline='red', width=3)

plt.figure(figsize=(10, 8))
plt.imshow(img)
plt.title(f"Category: {sample['name']}")
plt.axis('off')
plt.show()

In [ ]:
import numpy as np

In [ ]:
def rescale_bounding_boxes(df, target_width, target_height, bbox_column='bbox',
                           width_column='width', height_column='height',
                           output_column='bbox_scaled'):
    """
    Reescala bounding boxes a nuevas dimensiones objetivo.
    
    Parameters:
    - df: pandas.DataFrame con columnas bbox, width, height.
    - target_width: nuevo ancho de la imagen.
    - target_height: nuevo alto de la imagen.
    - bbox_column: nombre de la columna que contiene los bboxes originales.
    - width_column: nombre de la columna que contiene el ancho original.
    - height_column: nombre de la columna que contiene el alto original.
    - output_column: nombre de la columna de salida que guardará las cajas reescaladas.
    
    Returns:
    - DataFrame con columna adicional de bboxes reescalados.
    """
    original_width = df[width_column].iloc[0]
    original_height = df[height_column].iloc[0]
    scale_x = target_width / original_width
    scale_y = target_height / original_height
    
    def scale_bbox(bbox):
        if bbox is None or isinstance(bbox, float) and np.isnan(bbox):
            return None 
        if not isinstance(bbox, (list, tuple)) or len(bbox) != 4:
            return None 
        x0, y0, width, height = map(float, bbox)
        x1, y1 = x0 + width, y0 + height
        x0_scaled = x0 * scale_x
        y0_scaled = y0 * scale_y
        width_scaled = (x1 - x0) * scale_x
        height_scaled = (y1 - y0) * scale_y
        return [x0_scaled, y0_scaled, width_scaled, height_scaled]

    df[output_column] = df[bbox_column].apply(scale_bbox)
    return df

In [ ]:
rescaled_df = rescale_bounding_boxes(merged_df, target_width=1024, target_height=747)
print(rescaled_df[['file_name', 'bbox_scaled']].head())

In [ ]:
def show_random_image_with_bbox(df, images_folder, bbox_column='bbox_scaled', 
                                file_column='file_name', label_column='name'):
    """
    Muestra aleatoriamente una imagen del dataframe con su bbox dibujado.
    
    Parameters:
    - df: pandas.DataFrame que debe tener bbox escalados.
    - images_folder: ruta a la carpeta donde están las imágenes.
    - bbox_column: columna donde está el bbox escalado.
    - file_column: columna con el nombre del archivo de imagen.
    - label_column: columna con la etiqueta/clase.
    """
    valid_df = df[df[bbox_column].notnull()]

    sample = valid_df.sample(1).iloc[0]
    image_path = os.path.join(images_folder, sample[file_column])

    img = Image.open(image_path)
    draw = ImageDraw.Draw(img)

    x0, y0, width, height = map(float, sample[bbox_column])
    x1, y1 = x0 + width, y0 + height

    draw.rectangle([x0, y0, x1, y1], outline='red', width=3)

    plt.figure(figsize=(8, 6))
    plt.imshow(img)
    plt.title(f"Category: {sample[label_column]}")
    plt.axis('off')
    plt.show()

In [ ]:
show_random_image_with_bbox(rescaled_df, images_folder=IMAGES_FOLDER)

In [ ]:
rescaled_df[rescaled_df['bbox_scaled'].notnull()]['file_name'].nunique()

In [ ]:
bbox_counts = rescaled_df[rescaled_df['bbox_scaled'].notnull()].groupby('file_name').size()
bbox_counts.shape

In [ ]:
multi_bbox_images = bbox_counts[bbox_counts > 1]
num_multi_bbox_images = len(multi_bbox_images)

print(f"Número de imágenes con más de un bbox: {num_multi_bbox_images}")

In [ ]:
multi_bbox_images.head()

In [ ]:
import random

In [ ]:
def show_image_with_multi_bbox(df, images_folder, file_name_column='file_name',
                               bbox_column='bbox_scaled', label_column='name', file_name=None):
    """
    Muestra una imagen (aleatoria si no se especifica) con todos sus bboxes dibujados.
    
    Parameters:
    - df: pandas.DataFrame con bboxes escalados.
    - images_folder: ruta a la carpeta de imágenes.
    - file_name_column: columna con el nombre del archivo.
    - bbox_column: columna con las cajas escaladas.
    - label_column: columna con las etiquetas.
    - file_name: (opcional) nombre de archivo específico. Si None, selecciona aleatorio.
    """
    valid_df = df[df[bbox_column].notnull()]
    
    if file_name is None:
        multi_bbox_files = valid_df.groupby(file_name_column).size()
        multi_bbox_files = multi_bbox_files[multi_bbox_files > 1].index
        if len(multi_bbox_files) == 0:
            print("No hay imágenes con múltiples bboxes.")
            return
        file_name = random.choice(multi_bbox_files)
    
    image_df = valid_df[valid_df[file_name_column] == file_name]
    
    if image_df.empty:
        print(f"No se encontraron bboxes para la imagen {file_name}")
        return
    
    image_path = os.path.join(images_folder, file_name)
    img = Image.open(image_path)
    draw = ImageDraw.Draw(img)
    
    for _, row in image_df.iterrows():
        x0, y0, width, height = map(float, row[bbox_column])
        x1, y1 = x0 + width, y0 + height
        draw.rectangle([x0, y0, x1, y1], outline='red', width=3)
        draw.text((x0, y0), row[label_column], fill='red')
    
    plt.figure(figsize=(10, 8))
    plt.imshow(img)
    plt.title(f"File: {file_name}")
    plt.axis('off')
    plt.show()

In [ ]:
show_image_with_multi_bbox(rescaled_df, images_folder=IMAGES_FOLDER, file_name='585f4e71-23d2-11e8-a6a3-ec086b02610b.jpg')

In [ ]:
show_image_with_multi_bbox(rescaled_df, images_folder=IMAGES_FOLDER)

## ¿Cuántos ejemplos por clase tenemos? (bbox = clase detectada)

In [ ]:
valid_df = rescaled_df[rescaled_df['bbox_scaled'].notnull()]
class_counts = valid_df['name'].value_counts()
class_counts

| Clase (inglés) | Clase (español) | Cantidad |
| -------------- | --------------- | -------- |
| opossum        | zarigüeya       | 2,514    |
| rabbit         | conejo          | 2,278    |
| coyote         | coyote          | 1,371    |
| cat            | gato            | 1,170    |
| squirrel       | ardilla         | 1,037    |
| raccoon        | mapache         | 1,030    |
| dog            | perro           | 769      |
| bobcat         | lince rojo      | 684      |
| car            | auto/coche      | 668      |
| bird           | ave             | 560      |
| rodent         | roedor          | 264      |
| skunk          | mofeta/zorrillo | 214      |
| deer           | ciervo/venado   | 44       |
| badger         | tejón           | 9        |
| fox            | zorro           | 5        |


### Análisis de balance de clases
- Se evidencian clases muy minoritarias (fox, badger, deer, etc)
- Se podria utilizar *data augmentation* para clases medianas, y few-shot para ultra minoritarias
- Tambien su pudo haber agrupado clases, si es que tuviera sentido biológico o justificación científica, cosas que se duda entre zorro, tejón y ciervo.

Decisión:
- Para clases de ≥200 ejemplos, aplicar augmentation
- Para clases de <50 se descartan temporalmente, son demasiado escasas

In [ ]:
rescaled_df.head()

In [ ]:
rescaled_df.shape

### *data augmentation*

In [ ]:
def save_recortes_by_class(df, images_folder, output_folder, bbox_column='bbox_scaled',
                           file_column='file_name', label_column='name'):
    """
    Recorta las imágenes según bbox y guarda en carpetas por clase.
    """
    for _, row in df.iterrows():
        class_name = row[label_column]
        bbox = row[bbox_column]
        if bbox is None:
            continue
        file_name = row[file_column]
        
        class_folder = os.path.join(output_folder, class_name)
        os.makedirs(class_folder, exist_ok=True)
        
        img_path = os.path.join(images_folder, file_name)
        img = Image.open(img_path)
        
        x0, y0, width, height = map(float, bbox)
        x1, y1 = x0 + width, y0 + height
        cropped_img = img.crop((x0, y0, x1, y1))
        
        save_name = f"{os.path.splitext(file_name)[0]}_{row['id_ann']}.jpg"
        cropped_img.save(os.path.join(class_folder, save_name))

In [ ]:
OUTPUT_FOLDER = os.path.join('bboxes', 'bboxes_recortes')

In [ ]:
OUTPUT_FOLDER

In [ ]:
save_recortes_by_class(rescaled_df, IMAGES_FOLDER, OUTPUT_FOLDER)

In [ ]:
import albumentations as A
import cv2
from tqdm import tqdm

In [ ]:
def apply_augmentations_for_class(class_name, input_root, output_root, num_augmentations=2):
    """
    Aplica augmentations a una clase específica.
    
    Parameters:
    - class_name: nombre de la clase (carpeta dentro de input_root).
    - input_root: carpeta raíz de los recortes originales.
    - output_root: carpeta raíz para guardar augmentations.
    - num_augmentations: cuántas imágenes augmentadas generar por original.
    """
    input_folder = os.path.join(input_root, class_name)
    output_folder = os.path.join(output_root, class_name)
    
    transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.2),
        A.Rotate(limit=20, p=0.5),
        A.GaussNoise(p=0.2)
    ])
    
    os.makedirs(output_folder, exist_ok=True)
    
    for img_name in tqdm(os.listdir(input_folder), desc=f"Augmenting {class_name}"):
        img_path = os.path.join(input_folder, img_name)
        img = cv2.imread(img_path)
        
        if img is None:
            print(f"⚠️ No se pudo leer la imagen: {img_path}")
            continue
        
        for i in range(num_augmentations):
            augmented = transform(image=img)['image']
            save_name = f"{os.path.splitext(img_name)[0]}_aug{i}.jpg"
            cv2.imwrite(os.path.join(output_folder, save_name), augmented)


In [ ]:
INPUT_FOLDER = os.path.join('bboxes', 'bboxes_recortes')
AUG_OUTPUT_FOLDER = os.path.join('bboxes', 'augmented')

In [ ]:
apply_augmentations_for_class('dog', INPUT_FOLDER, AUG_OUTPUT_FOLDER, num_augmentations=2)
apply_augmentations_for_class('bobcat', INPUT_FOLDER, AUG_OUTPUT_FOLDER, num_augmentations=2)
apply_augmentations_for_class('car', INPUT_FOLDER, AUG_OUTPUT_FOLDER, num_augmentations=2)
apply_augmentations_for_class('bird', INPUT_FOLDER, AUG_OUTPUT_FOLDER, num_augmentations=2)
apply_augmentations_for_class('rodent', INPUT_FOLDER, AUG_OUTPUT_FOLDER, num_augmentations=4)
apply_augmentations_for_class('skunk', INPUT_FOLDER, AUG_OUTPUT_FOLDER, num_augmentations=4)

In [ ]:
def show_random_augmented_image(class_name, augmented_folder='bboxes\\augmented'):
    """
    Muestra aleatoriamente una imagen augmentada de la clase indicada.
    
    Parameters:
    - class_name: nombre de la carpeta/clase dentro de augmented_folder.
    - augmented_folder: carpeta raíz donde están las carpetas de augmentations.
    """
    class_folder = os.path.join(augmented_folder, class_name)
    if not os.path.exists(class_folder):
        print(f"⚠️ Carpeta no encontrada: {class_folder}")
        return
    
    images = os.listdir(class_folder)
    if not images:
        print(f"⚠️ No hay imágenes en {class_folder}")
        return
    
    selected_image = random.choice(images)
    img_path = os.path.join(class_folder, selected_image)
    
    img = Image.open(img_path)
    plt.figure(figsize=(6, 6))
    plt.imshow(img)
    plt.title(f"Class: {class_name}\nFile: {selected_image}")
    plt.axis('off')
    plt.show()

In [ ]:
show_random_augmented_image('dog')

### Reflexión

**Lo ideal sería aplicar augmentations únicamente al objeto (solo al perro o animal en cuestión) y no al fondo.**

Para lograr esto, no basta con trabajar sobre bounding boxes...

Este nivel de precisión requiere:

- Máscaras de segmentación pixel a pixel del animal dentro de la imagen.
- Modelos especializados como U-Net u otras arquitecturas de segmentación.

El enfoque es válido, pero es importante reconocer sus limitaciones.

## Armar dataloader

In [ ]:
import shutil
import os

In [ ]:
dataset_dir = os.path.join('data', 'dataloader')

bboxes_recortes_dir = os.path.join('bboxes', 'bboxes_recortes')
augmented_dir = os.path.join('bboxes', 'augmented')

In [ ]:
allowed_classes = [
    'opossum', 'rabbit', 'coyote', 'cat', 'squirrel', 'raccoon', 
    'dog', 'bobcat', 'car', 'bird', 'rodent', 'skunk'
    # NOTA: estamos excluyendo deer, badger, fox
]

Encapsulamos únicamente las clases que tienen una muestra representativa

In [ ]:
os.makedirs(dataset_dir, exist_ok=True)

for class_name in allowed_classes:
    dest_class_dir = os.path.join(dataset_dir, class_name)
    os.makedirs(dest_class_dir, exist_ok=True)

    # Copiar desde bboxes_recortes
    src_bbox_class_dir = os.path.join(bboxes_recortes_dir, class_name)
    if os.path.exists(src_bbox_class_dir):
        for filename in tqdm(os.listdir(src_bbox_class_dir), desc=f"{class_name} - bbox", leave=True):
            src_file = os.path.join(src_bbox_class_dir, filename)
            dest_file = os.path.join(dest_class_dir, filename)
            shutil.copy2(src_file, dest_file)

    # Copiar desde augmented
    src_aug_class_dir = os.path.join(augmented_dir, class_name)
    if os.path.exists(src_aug_class_dir):
        for filename in tqdm(os.listdir(src_aug_class_dir), desc=f"{class_name} - aug", leave=True):
            src_file = os.path.join(src_aug_class_dir, filename)
            dest_file = os.path.join(dest_class_dir, filename)
            shutil.copy2(src_file, dest_file)


In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)
